In [1]:
import pandas as pd
import json

with open('../results.json', 'r') as f:
    results_dict = json.load(f)

decimal_precision = 2

In [2]:
common_columns = ['flow', 'sampler', 'second_moment_squared_bias']

tmp1 = pd.DataFrame(results_dict['dimensionality_ablation'])[common_columns + ['dimensionality']]
tmp2 = pd.DataFrame(results_dict['curvature_strength_ablation'])[common_columns + ['curvature_strength']]
tmp3 = pd.DataFrame(results_dict['weight_ablation'])[common_columns + ['weight_scale']]
tmp4 = pd.DataFrame(results_dict['n_components_ablation'])[common_columns + ['n_components']]

tmps = []
for i, t in enumerate([tmp1, tmp2, tmp3, tmp4]):
    t['method'] = t['sampler'] + t[t.columns[-1]].map(str) + f'_{i}'
    t = t.loc[~t['flow'].isin(['ot-flow', 'p-resflow', 'proximal-resflow'])]
    t = t.drop([t.columns[-2]], axis=1)
    tmps.append(t)
tmps.append(pd.concat(tmps, axis=0))

tmps = [t.loc[t['flow'] != 'None'] for t in tmps]
tmps_jumps = [t[~t['sampler'].isin(['neutra_hmc', 'neutra_mh'])].drop('sampler', axis=1) for t in tmps]
tmps_neutra = [t[t['sampler'].isin(['neutra_hmc', 'neutra_mh'])].drop('sampler', axis=1) for t in tmps]

In [3]:
from notebooks.util import standardized_rank, make_bold_mask, to_booktabs_table, get_standard_flow_order, FLOW_PRETTY_MATH

_fr = pd.concat([
    standardized_rank(t, rank_what='flow', rank_across='method').apply(lambda el: round(el, decimal_precision), axis=1).apply(tuple, axis=1)
    for t in tmps_jumps
], axis=1)
_fr.columns = ['Dimensionality', 'Curvature', 'Mode weight', 'Components', 'All']
order = get_standard_flow_order([f for f, in _fr.index])
_fr = _fr.iloc[order]
_fr.index = [FLOW_PRETTY_MATH[s] for (s,) in _fr.index]
bold_mask = make_bold_mask(_fr, top_quantile=0.8)
_fr = _fr.reset_index()
_fr.columns = ['NF'] + list(_fr.columns)[1:]

to_booktabs_table(
    _fr,
    bold_mask=bold_mask,
    caption=r'$\overline{r} \pm \hat{\sigma}$ for NFs in Jump MCMC given NF scalability scores when varying target properties: dimensionality, curvature strength, variance of mode weights, number of modes. NFs with $\overline{r}$ in the 20th percentile are shown in bold. Ranks computed separately for each target property.',
    save_to_file='tables/target-variations-jump-mcmc.tex',
    label='tab:target-variations-jump-mcmc'
)

\begin{table}

            \renewrobustcmd{\bfseries}{\fontseries{b}\selectfont}
            \renewrobustcmd{\boldmath}{}
            \sisetup{%
                table-align-uncertainty=true,
                detect-all,
                separate-uncertainty=true,
                mode=text,
                round-mode=uncertainty,
                round-precision=2,
                table-format = 2.2(2),
                table-column-width=2.1cm
            }
            
\begin{tabular}{l
S
S
S
S
S}
\toprule
{NF} & {Dimensionality} & {Curvature} & {Mode weight} & {Components} & {All} \\
\midrule
{NICE} & \bfseries -0.68(0.24) & \bfseries -0.48(0.27) & -0.26(0.19) & \bfseries -0.34(0.23) & \bfseries -0.43(0.11) \\
{Real NVP} & -0.13(0.24) & \bfseries -0.41(0.28) & -0.06(0.17) & -0.18(0.23) & -0.18(0.11) \\
{C-LR-NSF} & -0.27(0.17) & \bfseries -1.01(0.14) & \bfseries -0.45(0.27) & \bfseries -0.34(0.19) & \bfseries -0.49(0.11) \\
{C-RQ-NSF} & \bfseries -0.68(0.21) & -0.39(0.17) & -0.31(0.18) &

In [4]:
from util import make_bold_mask, to_booktabs_table
from util import get_standard_flow_order, FLOW_PRETTY_MATH

decimal_precision = 2

_fr = pd.concat([
    standardized_rank(t, rank_what='flow', rank_across='method').apply(lambda el: round(el, decimal_precision), axis=1).apply(tuple, axis=1)
    for t in tmps_neutra
], axis=1)
_fr.columns = ['Dimensionality', 'Curvature', 'Mode weight', 'Components', 'All']
order = get_standard_flow_order([f for f, in _fr.index])
_fr = _fr.iloc[order]
_fr.index = [FLOW_PRETTY_MATH[s] for (s,) in _fr.index]
bold_mask = make_bold_mask(_fr, top_quantile=0.8)
_fr = _fr.reset_index()
_fr.columns = ['NF'] + list(_fr.columns)[1:]

to_booktabs_table(
    _fr,
    bold_mask=bold_mask,
    caption=r'$\overline{r} \pm \hat{\sigma}$ for NFs in NeuTra MCMC given NF scalability scores when varying target properties: dimensionality, curvature strength, variance of mode weights, number of modes. NFs with $\overline{r}$ in the 20th percentile are shown in bold. Ranks computed separately for each target property.',
    save_to_file='tables/target-variations-neutra.tex',
    label='tab:target-variations-neutra'
)

\begin{table}

            \renewrobustcmd{\bfseries}{\fontseries{b}\selectfont}
            \renewrobustcmd{\boldmath}{}
            \sisetup{%
                table-align-uncertainty=true,
                detect-all,
                separate-uncertainty=true,
                mode=text,
                round-mode=uncertainty,
                round-precision=2,
                table-format = 2.2(2),
                table-column-width=2.1cm
            }
            
\begin{tabular}{l
S
S
S
S
S}
\toprule
{NF} & {Dimensionality} & {Curvature} & {Mode weight} & {Components} & {All} \\
\midrule
{NICE} & 0.01(0.14) & -0.38(0.33) & \bfseries -0.93(0.15) & -0.42(0.31) & -0.46(0.13) \\
{Real NVP} & 0.05(0.22) & \bfseries -0.85(0.26) & -0.62(0.21) & 0.10(0.22) & -0.32(0.13) \\
{C-LR-NSF} & \bfseries -0.60(0.14) & -0.44(0.38) & \bfseries -1.16(0.10) & -0.47(0.30) & \bfseries -0.70(0.12) \\
{C-RQ-NSF} & 0.29(0.17) & \bfseries -0.74(0.53) & \bfseries -1.27(0.11) & -0.39(0.30) & \bfseries -0.53(0.1